<a href="https://colab.research.google.com/github/caoscott/cifar10/blob/master/cifar10_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchvision import transforms

train_preprocess = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [2]:
from torchvision import datasets

cifar10_train = datasets.CIFAR10('data', train=True, download=True,
                                 transform=train_preprocess)
cifar10_test = datasets.CIFAR10('data', train=False, download=True,
                                transform=test_preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
from torch.utils import data

train_loader = data.DataLoader(cifar10_train, batch_size=512, 
                               num_workers=3, shuffle=True, drop_last=True)
test_loader = data.DataLoader(cifar10_test, batch_size=2048, 
                              num_workers=3, shuffle=False, drop_last=False)

In [0]:
from torch import nn

class Pass(nn.Module):
  def forward(self, x):
    return x

In [0]:
from torchvision import models

model = models.resnet152()
# model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
model.maxpool = Pass()
model.fc = nn.Linear(2048, 10)
model = model.cuda()

In [0]:
from torch import optim
from torch.optim import lr_scheduler

loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,
                      weight_decay=5e-4, nesterov=True)
lr_schedule = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=10, 
                                             verbose=True)

In [0]:
def acc(loader):
  with torch.no_grad():
    total = 0
    correct = 0
    for x, y in loader:
      x, y = x.cuda(), y.cuda()
      scores = model(x)
      _, predicted = torch.max(scores, 1)
      total += y.size(0)
      correct += predicted.eq(y).sum().item()
    return correct/total

In [8]:
import time
import torch

for epoch in range(0, 100):
  batch_time = time.time()
  epoch_time = batch_time
  for iteration, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    x, y = x.cuda(), y.cuda()
    scores = model(x)
    loss = loss_fn(scores, y)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
      _, predicted = torch.max(scores, 1)
      train_acc = torch.eq(predicted, y).sum().item()/x.shape[0]

      print('\r| Epoch {:3d} Iter {:3d} Time {:.3f} Avg Time {:.3f}\t'
            'Batch Loss: {:.4f} Batch Acc {:.3f}'.format(
                epoch, iteration+1, time.time()-batch_time, 
                (time.time()-epoch_time)/(iteration+1),
                loss.item(), train_acc
            ), end='')
      batch_time = time.time()
      del x, y, scores, loss, predicted
  optimizer.zero_grad()
  test_acc = acc(test_loader)
  lr_schedule.step(test_acc)
  print('\n| Epoch {:3d} Time {:.3f} \tTest Acc {:.3f}'.format(
    epoch, time.time()-epoch_time, test_acc
  ))

| Epoch   0 Iter  97 Time 1.267 Avg Time 1.274	Batch Loss: 2.3084 Batch Acc 0.113
| Epoch   0 Time 131.935 	Test Acc 0.121
| Epoch   1 Iter  97 Time 1.255 Avg Time 1.266	Batch Loss: 2.0116 Batch Acc 0.219
| Epoch   1 Time 131.063 	Test Acc 0.208
| Epoch   2 Iter  97 Time 1.261 Avg Time 1.268	Batch Loss: 1.9492 Batch Acc 0.232
| Epoch   2 Time 131.144 	Test Acc 0.275
| Epoch   3 Iter  97 Time 1.263 Avg Time 1.267	Batch Loss: 1.6992 Batch Acc 0.334
| Epoch   3 Time 131.388 	Test Acc 0.334
| Epoch   4 Iter  97 Time 1.265 Avg Time 1.269	Batch Loss: 1.5954 Batch Acc 0.416
| Epoch   4 Time 131.396 	Test Acc 0.399
| Epoch   5 Iter  97 Time 1.263 Avg Time 1.271	Batch Loss: 1.5264 Batch Acc 0.441
| Epoch   5 Time 131.730 	Test Acc 0.427
| Epoch   6 Iter  97 Time 1.261 Avg Time 1.271	Batch Loss: 1.5779 Batch Acc 0.424
| Epoch   6 Time 131.942 	Test Acc 0.472
| Epoch   7 Iter  97 Time 1.259 Avg Time 1.272	Batch Loss: 1.3906 Batch Acc 0.480
| Epoch   7 Time 131.603 	Test Acc 0.502
| Epoch   8 Iter

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)


KeyboardInterrupt: ignored

In [0]:
torch.cuda.empty_cache()

In [13]:
!nvidia-smi

Sun May 19 07:24:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |   2703MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
import time
import torch

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,
                      weight_decay=5e-4, nesterov=True)
lr_schedule = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=10, 
                                             verbose=True)

for epoch in range(0, 100):
  batch_time = time.time()
  epoch_time = batch_time
  for iteration, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    x, y = x.cuda(), y.cuda()
    scores = model(x)
    loss = loss_fn(scores, y)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
      _, predicted = torch.max(scores, 1)
      train_acc = torch.eq(predicted, y).sum().item()/x.shape[0]

      print('\r| Epoch {:3d} Iter {:3d} Time {:.3f} Avg Time {:.3f}\t'
            'Batch Loss: {:.4f} Batch Acc {:.3f}'.format(
                epoch, iteration+1, time.time()-batch_time, 
                (time.time()-epoch_time)/(iteration+1),
                loss.item(), train_acc
            ), end='')
      batch_time = time.time()
      del x, y, scores, loss, predicted
  optimizer.zero_grad()
  test_acc = acc(test_loader)
  lr_schedule.step(test_acc)
  print('\n| Epoch {:3d} Time {:.3f} \tTest Acc {:.3f}'.format(
    epoch, time.time()-epoch_time, test_acc
  ))

| Epoch   0 Iter  97 Time 1.263 Avg Time 1.234	Batch Loss: 0.4823 Batch Acc 0.824
| Epoch   0 Time 128.153 	Test Acc 0.785
| Epoch   1 Iter   7 Time 1.261 Avg Time 1.336	Batch Loss: 0.4643 Batch Acc 0.836

KeyboardInterrupt: ignored